In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings

In [14]:
# !curl -fsSL https://ollama.com/install.sh | sh
# !export OLLAMA_HOST=localhost:8888
# !ollama serve
# !ollama pull mxbai-embed-large

>>> Downloading ollama...
######################################################################## 100.0%#######                       71.8%
>>> Installing ollama to /usr/local/bin...
>>> Adding ollama user to render group...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> Enabling and starting ollama service...
>>> NVIDIA GPU installed.
Error: listen tcp 127.0.0.1:11434: bind: address already in use
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling 819c2adf5ce6... 100% ▕████████████████▏ 669 MB                         
pulling c71d239df917... 100% ▕████████████████▏  11 KB                         
pulling b837481ff855... 100% ▕████████████████▏   16 B                         
pulling 38badd946f91... 100% ▕████████████████▏  408 B                         
verifying sha256 digest 
writing manifest 
removing any unused layers 
success 


In [23]:
if __name__=="__main__":
        from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
        DB_FAISS_PATH = '../vectorstore'
        DATA_PATH = "random machine learning.pdf"
        loader = DirectoryLoader('../data', glob="*.md")
        docs = loader.load()
        print(len(docs))
        # text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        # splits = text_splitter.split_documents(docs)
        # vectorstore = FAISS.from_documents(documents=splits, embedding=OllamaEmbeddings(model="mxbai-embed-large", show_progress=True))
        # vectorstore.save_local(DB_FAISS_PATH)
        

        
        # DB_FAISS_PATH = '../vectorstore'
        # loader=PyPDFLoader("random machine learning.pdf")
        # docs=loader.load()
        # text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        # splits = text_splitter.split_documents(docs)
        # vectorstore = FAISS.from_documents(documents=splits, embedding=OllamaEmbeddings(model="mxbai-embed-large", show_progress=True))
        # vectorstore.save_local(DB_FAISS_PATH)

0
